### 필요한 라이브러리 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D

'''#GPU 자원이 부족한 경우 아래 코드를 이용하세요
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)'''

'#GPU 자원이 부족한 경우 아래 코드를 이용하세요\nfrom tensorflow.compat.v1 import ConfigProto\nfrom tensorflow.compat.v1 import InteractiveSession\n\nconfig = ConfigProto()\nconfig.gpu_options.allow_growth = True\nsession = InteractiveSession(config=config)'

### 데이터 전처리

In [ ]:
w_list = sorted(glob("/content/drive/MyDrive/Study/Dacon/dam/water_data/*.csv"))
w_list

['/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2012.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2013.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2014.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2015.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2016.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2017.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2018.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2019.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2020.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2021.csv',
 '/content/drive/MyDrive/Study/Dacon/dam/water_data/data_2022.csv']

In [ ]:
pd.read_csv(w_list[0]).shape

(26496, 15)

In [ ]:
pd.read_csv(w_list[0]).head(4)

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630
0,2012-05-01 00:00,24.800,555.0,219.07,24.93,555.0,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18
1,2012-05-01 00:10,24.794,464.6,218.86,25.15,562.9,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18
2,2012-05-01 00:20,24.789,478.1,218.69,25.31,576.4,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18
3,2012-05-01 00:30,24.789,464.8,218.69,25.31,563.1,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17


In [ ]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
    
    tmp = pd.read_csv(i)
    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)
    
    for j in tqdm(range(len(tmp)-432)):
        train_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                       "tototf", "tide_level",
                                                       "fw_1018662", "fw_1018680",
                                                       "fw_1018683", "fw_1019630"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:27<00:00, 952.95it/s]


In [ ]:
train_data = np.array(train_data)
train_label = np.array(train_label)

print(train_data.shape)
print(train_label.shape)

(260640, 432, 10)
(260640, 4)


### 모델링 및 모델 학습

In [ ]:
input_shape = (train_data[0].shape[0], train_data[0].shape[1])

model = Sequential()
model.add(GRU(512, input_shape=input_shape))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(4, activation = 'relu'))

optimizer = tf.optimizers.RMSprop(0.0005)

model.compile(optimizer=optimizer,loss='mse', metrics=['mae'])

In [ ]:
model.fit(train_data, train_label, epochs=15, batch_size=256)

Epoch 1/15
1019/1019 [==============================] - 131s 125ms/step - loss: 42418.2422 - mae: 180.9142
Epoch 2/15
1019/1019 [==============================] - 128s 126ms/step - loss: 2841.6448 - mae: 23.4947
Epoch 3/15
1019/1019 [==============================] - 128s 125ms/step - loss: 683.0254 - mae: 8.7152
Epoch 4/15
1019/1019 [==============================] - 128s 125ms/step - loss: 380.7025 - mae: 6.9864
Epoch 5/15
1019/1019 [==============================] - 127s 125ms/step - loss: 266.5389 - mae: 6.0927
Epoch 6/15
1019/1019 [==============================] - 127s 125ms/step - loss: 204.2237 - mae: 5.6130
Epoch 7/15
1019/1019 [==============================] - 127s 125ms/step - loss: 165.5057 - mae: 5.2296
Epoch 8/15
1019/1019 [==============================] - 127s 125ms/step - loss: 147.8658 - mae: 5.0149
Epoch 9/15
1019/1019 [==============================] - 127s 125ms/step - loss: 135.6986 - mae: 4.8476
Epoch 10/15
1019/1019 [==============================] - 127s 125ms

### 추론 데이터셋 만들기

In [ ]:
test_data = []
test_label = []

tmp = pd.read_csv(w_list[-1])
tmp = tmp.replace(" ", np.nan)
# 이전값을 사용
tmp = tmp.fillna(method = 'pad')
tmp = tmp.fillna(0)
    
#tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]] = tmp.loc[:, ["wl_1018662", "wl_1018680", "wl_1018683", "wl_1019630"]]*100
    
for j in tqdm(range(4032, len(tmp)-432)):
    test_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                    "tototf", "tide_level",
                                                    "fw_1018662", "fw_1018680",
                                                    "fw_1018683", "fw_1019630"]]).astype(float))
        
    test_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                    "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 6912/6912 [00:07<00:00, 951.51it/s]


In [ ]:
test_data = np.array(test_data)
test_label = np.array(test_label)

print(test_data.shape)
print(test_label.shape)

(6912, 432, 10)
(6912, 4)


### 제출 파일 만들기

In [ ]:
pred = model.predict(test_data)

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Study/Dacon/dam/sample_submission.csv")

sample_submission["wl_1018662"] = pred[0]
sample_submission["wl_1018680"] = pred[1]
sample_submission["wl_1018683"] = pred[2]
sample_submission["wl_1019630"] = pred[3]

In [ ]:
sample_submission.to_csv("submission.csv", index = False)